Copyright (c) Microsoft Corporation.

Licensed under the MIT License.
#### ------------------------------------

# SEND EMAIL AGENT

This sample demonstrates how to use basic agent operations from the Azure Agents service using a synchronous client and, in addition, calls a dedicated `send_email` function (from `email_services.py`) to send an email.

## Instructions

### Before Running the Sample

Install the required packages:

```bash
pip install azure-ai-projects azure-identity
```

Set these environment variables with your own values:

- **PROJECT_CONNECTION_STRING**  
  The project connection string, as found in the overview page of your Azure AI Foundry project.

- **MODEL_DEPLOYMENT_NAME**  
  The deployment name of the AI model, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.

Additionally, the `email_services.py` module should handle the following environment variables:

- `EMAIL_COMMUNICATION_SERVICES_STRING`
- `RECIPIENT_EMAIL`
- `SENDER_EMAIL`


In [5]:
import os
import time
import json
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    FunctionTool,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
    MessageTextContent,
)
from azure.identity import DefaultAzureCredential
from email_services import azure_send_email 

In [ ]:
def send_email_function(subject: str, body: str) -> dict:
    """
    Wrapper function for sending an email.
    
    Expects a single parameter:
        - subject (str): The email subject.
    
    The function constructs the email using a default template where only the subject
    value is replaced, while all other email details remain unchanged.
    """
    return azure_send_email(subject, body)

send_email_function.__name__ = "send_email"

# Create the function tool with a set of functions.
functions = FunctionTool(functions={send_email_function})

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="email-assistant",
        instructions=(
            """
            You are an assistant that sends emails using the send_email function.
            When a request to send an email is received, call the send_email tool and confirm that the email has been sent.
            The function expects two parameters: 'subject' and 'body
            Use the following format as a guide:
            
            {
                "subject": "<New Email Subject>",
                "body": "<Email Body>"
            }
            
            The 'subject' and 'body' value will be provided by the user.
            """
        ),
        tools=functions.definitions,  # Register our custom tool with the agent.
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a conversation thread.
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create a user message instructing the assistant to send an email.
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            "Please send an email with subject 'This is Azure AI Agent test' and the body 'This is a test email to check the Azure AI Agent'."
        ),
    )
    print(f"Created message, ID: {message.id}")

    # Start the agent run.
    run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Created run, ID: {run.id}")

    # Poll for run status and check for tool call requirements.
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        # When the run requires a tool call, process the custom email function.
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=json.dumps(output),
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    for data_point in reversed(messages.data):
        if data_point.content and isinstance(data_point.content[-1], MessageTextContent):
            print(f"{data_point.role}: {data_point.content[-1].text.value}")
    print(f"Messages: {messages}")

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


Created agent, ID: asst_TbIWYnRCwH2Yj6XIuumk1odc
Created thread, ID: thread_nRCOzM8DnS1riH75766aEjnq
Created message, ID: msg_6ltyC5QGUuOMJxhrFwBWAZc3
Created run, ID: run_4WogyUpK9ZAEbVLZm6QPjKdS
Current run status: in_progress
Executing tool call: {'id': 'call_johWGQKK5Sk0yBMGn3lITQnB', 'type': 'function', 'function': {'name': 'send_email', 'arguments': '{"subject":"This is Azure AI Agent test","body":"This is a test email to check the Azure AI Agent"}'}}
Email send poller status: Running
Successfully sent the email (operation id: 62e3787f-de2c-4929-a306-0cb4f2da7a6d)
Tool outputs: [{'tool_call_id': 'call_johWGQKK5Sk0yBMGn3lITQnB', 'output': '{"operationId": "62e3787f-de2c-4929-a306-0cb4f2da7a6d", "status": "Succeeded", "message": "Successfully sent the email (operation id: 62e3787f-de2c-4929-a306-0cb4f2da7a6d)"}'}]
Current run status: requires_action
Current run status: completed
Run completed with status: completed
user: Please send an email with subject 'This is Azure AI Agent tes